In [1]:
import mne
from mne import find_events
import numpy as np
import os
from mne.preprocessing import ICA, read_ica
import random 


In [2]:
#File paths
meg_MainFolder = "..\Data\MEG_Data\Data="
meg_raw_MainFolder = "..\Data\MEG_Data\RawData\\"
beh_MainFolder = "..\Data\Behavior_Data\\"
ica_MainFolder = "..\icaModels\\"

tmin, tmax = -0.1, 0.6
s_id = '33'

print('Subject id: ', s_id)
print('tmin = ', tmin)
print('tmax = ', tmax)

print('Raw Data folder: ', meg_raw_MainFolder)

filep = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', filep)

beh_folder = beh_MainFolder + 'P'+ s_id + "\\"
print('Bheavioral data folder: ', beh_folder)
ica_file = ica_MainFolder + 'S' + s_id + '\ica.fif'
print('Ica model folder: ', ica_file)

componentsToExclude  = [0,15]
print('ICA components to exclude: ', componentsToExclude)


Subject id:  33
tmin =  -0.1
tmax =  0.6
Raw Data folder:  ..\Data\MEG_Data\RawData\
Data folder:  ..\Data\MEG_Data\Data=-0.1_0.6\
Bheavioral data folder:  ..\Data\Behavior_Data\P33\
Ica model folder:  ..\icaModels\S33\ica.fif
ICA components to exclude:  [0, 15]


In [3]:
# Fixed variables

blocks = [0,1,2]

triggers_cry_real = [9, 10, 11] # By block: sound 1 + nu_consistent 
triggers_phone_real = [11, 12, 13] # By block: sound 3 + nu_consistent 

triggers_cry_omi = [14, 15, 16] # By block: sound 1 + nu_consistent + omission code (5)
triggers_phone_omi = [16, 17, 18] # By block: sound 3 + nu_consistent + omission code (5)
pred_all = [8, 9, 10]

congruency = [80, 90, 100]

# Initialize data variables
epoch_grad_all = []
epoch_mag_all = []

print('x---------------x')

pitch_V = len(blocks)*[None]
pitch_RT = len(blocks)*[None]
conf_V = len(blocks)*[None]
conf_RT = len(blocks)*[None]
resp_V = len(blocks)*[None]
corr_V = len(blocks)*[None]

x---------------x


In [4]:

# Check if ica model exists and load if there is one

if os.path.isfile(ica_file) is True:
    ica = read_ica(ica_file)
else:
    ica  = None
    print('No ica file!')
    

Reading ..\icaModels\S33\ica.fif ...
Now restoring ICA solution ...


<ipython-input-4-7382744846b8>:4: RuntimeWarning: This filename (..\icaModels\S33\ica.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica = read_ica(ica_file)


Ready.


In [5]:

all_conf = []
for bl in blocks:
    print('Block: ', bl)
    if int(s_id) > 22:
        if s_id == '38' or s_id == '39' or s_id == '40' or s_id == '41':
            filename = 'P' + s_id + '\\block_' +  str(bl) + '_tsss_mc_trans.fif'
            fraw = meg_raw_MainFolder + filename
            
        else:
            filename = 'S' + s_id + '\\Block_' + str(bl) + '_tsss_mc_trans.fif' # s_id +'_' + str(bl) OR 'block_' + str(bl) + '_tsss_mc_trans.fif' 
            fraw = meg_raw_MainFolder + filename
           
                
    else:
        filename = 'S' + s_id + '\\' + s_id + '_' + str(bl) + '_tsss_mc_trans.fif'
        fraw = meg_raw_MainFolder + filename
        
    
   
    if os.path.isfile(meg_raw_MainFolder) == True:
         print(fraw)
    else:
        print('No file named as ', fraw)
    
    
    if os.path.isfile(fraw):
        # read and filter raw data:
        raw = mne.io.read_raw_fif(fraw , preload = True)
        raw.info['ch_names'] = [chn.strip('.') for chn in raw.info['ch_names']]
        print("Data imported!")
        raw.filter(0.1, 40., fir_design='firwin')
        raw.load_data()
        raw_appliedICA = ica.apply(raw, exclude=componentsToExclude)
        
        # exctract events
        events101 = find_events(raw, 'STI101', shortest_event = 1,initial_event=True)
        if s_id == '14' or s_id == '13':
            events101[:,2] = events101[:,2] - 2048
            print("Adjusting triggers...")
        
        print("Events detected!")
        print('number of events: ', len(events101[:,2]))
        
        # Rename events so there is no overlap across blocks:
        pred_l = pred_all[bl]
        ind_resp, ind_epoch = 0, 0
        print(os.path.exists(beh_folder + 'Responses_S' + s_id + '_Block'+str(bl)+ '.npy'))
        
        if os.path.exists(beh_folder + 'Responses_S' + s_id + '_Block'+str(bl)+ '.npy'):
            pitch_V[bl], pitch_RT[bl], conf_V[bl], conf_RT[bl], resp_V[bl], corr_V[bl] = np.load(beh_folder + 'Responses_S' + s_id + '_Block'+str(bl)+ '.npy', allow_pickle = True)
            
            # Correct S26's triggers in the last block (100% level)
            if s_id == '26' and bl == 2:
                print('Adjusting P' + s_id + 's triggers ..')
                events101 = np.delete(events101, 0, axis=0)
                pitch_V[bl] = np.delete(pitch_V[bl], 0)
                pitch_RT[bl] = np.delete(pitch_RT[bl], 0)
                conf_V[bl] = np.delete(conf_V[bl], 0)
                conf_RT[bl] = np.delete(conf_RT[bl], 0)
                resp_V[bl] = np.delete(resp_V[bl], 0)
                corr_V[bl] = np.delete(corr_V[bl], 0)
                
                
            # keep only active responses which are kept as non-None in pitch_V
            active_resp_indices = np.where(pitch_V[bl] != None)[0]  
            pitch_V_active = [pitch_V[bl][i] for i in range(len(pitch_V[bl])) if i in active_resp_indices]
            corr_V_active = [corr_V[bl][i] for i in range(len(corr_V[bl])) if i in active_resp_indices]
            conf_V_active = [conf_V[bl][i] for i in range(len(conf_V[bl])) if i in active_resp_indices]
            
            #print(conf_V_active)
            #print(len(conf_V_active))
  
            for i in np.arange(len(events101)):
                
                # Real Sounds
                if events101[i,2] == triggers_cry_real[bl]:
                    events101[i,2] = (triggers_cry_real[bl] + 100*pred_l)*10 + 3
                    all_conf.append(None)
                
                elif events101[i,2] == triggers_phone_real[bl]:
                    events101[i,2] = (triggers_phone_real[bl] + 100*pred_l)*10 + 3
                    all_conf.append(None)

                
                # Omissions
                else:
                    if ind_resp < len(active_resp_indices):
                        if events101[i,2] == triggers_cry_omi[bl]:

                            if corr_V_active[ind_resp] == 1:
                                events101[i,2] = (triggers_cry_omi[bl] + 100*pred_l)*10 + 1
                                all_conf.append(conf_V_active[ind_resp])
                                ind_resp += 1

                            elif corr_V_active[ind_resp] == -1:
                                events101[i,2] = (triggers_cry_omi[bl] + 100*pred_l)*10
                                all_conf.append(conf_V_active[ind_resp])
                                ind_resp += 1

                            else:
                                events101[i,2] = (triggers_cry_omi[bl] + 100*pred_l)*10 + 2
                                all_conf.append(conf_V_active[ind_resp])
                                ind_resp += 1


                        elif events101[i,2] == triggers_phone_omi[bl]:

                            if corr_V_active[ind_resp] == 1:
                                #print('response is correct!')
                                events101[i,2] = (triggers_phone_omi[bl] + 100*pred_l)*10 + 1
                                all_conf.append(conf_V_active[ind_resp])
                                ind_resp += 1

                            elif corr_V_active[ind_resp] == -1:
                                events101[i,2] = (triggers_phone_omi[bl] + 100*pred_l)*10
                                all_conf.append(conf_V_active[ind_resp])
                                ind_resp += 1

                            else:
                                events101[i,2] = (triggers_phone_omi[bl] + 100*pred_l)*10 + 2
                                all_conf.append(conf_V_active[ind_resp])
                                ind_resp += 1
                    else:
                        print('Number of omissions and responses do not match!')
                                               
        
        event_id = {}

        # Generating new event ids  
        event_id['living_real_' + str(pred_l)] = (triggers_cry_real[bl] + 100*pred_l)*10 + 3
        event_id['living_omission_' + str(pred_l) + "_nores"] = (triggers_cry_omi[bl] + 100*pred_l)*10 + 2
        event_id['living_omission_' + str(pred_l) + "_corr"] = (triggers_cry_omi[bl] + 100*pred_l)*10 + 1
        event_id['living_omission_' + str(pred_l) + "_incorr"] = (triggers_cry_omi[bl] + 100*pred_l)*10
        
        event_id['object_real_'  + str(pred_l)] = (triggers_phone_real[bl] + 100*pred_l)*10 + 3
        event_id['object_omission_'  + str(pred_l) + "_nores"] = (triggers_phone_omi[bl] + 100*pred_l)*10 + 2
        event_id['object_omission_'  + str(pred_l) + "_corr"] = (triggers_phone_omi[bl] + 100*pred_l)*10 + 1
        event_id['object_omission_'  + str(pred_l) + "_incorr"] = (triggers_phone_omi[bl] + 100*pred_l)* 10
    
        # Cheking event ids if they exist in event list e.g. there might be no incorrect in 100% level
        if event_id['living_real_' + str(pred_l)] not in events101:
            del event_id['living_real_' + str(pred_l)]
        if event_id['object_real_' + str(pred_l)] not in events101:
            del event_id['object_real_' + str(pred_l)]
        if event_id['object_omission_'  + str(pred_l) + "_nores"] not in events101:
            del event_id['object_omission_'  + str(pred_l) + "_nores"]
        if event_id['object_omission_'  + str(pred_l) + "_corr"] not in events101:
            del event_id['object_omission_'  + str(pred_l) + "_corr"]
        if event_id['object_omission_'  + str(pred_l) + "_incorr"] not in events101:
            del event_id['object_omission_'  + str(pred_l) + "_incorr"]
        
        if event_id['living_omission_'  + str(pred_l) + "_nores"] not in events101:
            del event_id['living_omission_'  + str(pred_l) + "_nores"]
        if event_id['living_omission_'  + str(pred_l) + "_corr"] not in events101:
            del event_id['living_omission_'  + str(pred_l) + "_corr"]
        if event_id['living_omission_'  + str(pred_l) + "_incorr"] not in events101:
            del event_id['living_omission_'  + str(pred_l) + "_incorr"]
        

        # Generate Epoched data from raw data  
        #!!!! Be aware of detrend parameter!!
        epochs = mne.Epochs(raw_appliedICA, events=events101, event_id=event_id, baseline=None, tmin=tmin, tmax=tmax, preload=True, detrend=1)

        # Split to magnetometers and gradiometers:
        epoch_grad_all.append(epochs.copy().pick_types("grad"))
        epoch_mag_all.append(epochs.copy().pick_types("mag"))
        
        del epochs

        

Block:  0
No file named as  ..\Data\MEG_Data\RawData\S33\Block_0_tsss_mc_trans.fif
Opening raw data file ..\Data\MEG_Data\RawData\S33\Block_0_tsss_mc_trans.fif...
    Range : 122000 ... 1310999 =    122.000 ...  1310.999 secs
Ready.
Reading 0 ... 1188999  =      0.000 ...  1188.999 secs...


<ipython-input-5-f4267c4a09b9>:28: RuntimeWarning: This filename (..\Data\MEG_Data\RawData\S33\Block_0_tsss_mc_trans.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(fraw , preload = True)


Data imported!
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 sec)

Transforming to ICA space (33 components)
Zeroing out 2 ICA components
715 events found
Event IDs: [  1   8   9  10  11  14  16  74  76 104 106]
Events detected!
number of events:  715
True
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Not setting metadata
Not setting metadata
315 matching events found
No baseline cor

<ipython-input-5-f4267c4a09b9>:28: RuntimeWarning: This filename (..\Data\MEG_Data\RawData\S33\Block_1_tsss_mc_trans.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(fraw , preload = True)


Data imported!
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 sec)

Transforming to ICA space (33 components)
Zeroing out 2 ICA components
716 events found
Event IDs: [  1   9  10  11  12  15  16  17  75  77 105 107]
Events detected!
number of events:  716
True
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Not setting metadata
Not settin

<ipython-input-5-f4267c4a09b9>:28: RuntimeWarning: This filename (..\Data\MEG_Data\RawData\S33\Block_2_tsss_mc_trans.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(fraw , preload = True)


Data imported!
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 sec)

Transforming to ICA space (33 components)
Zeroing out 2 ICA components
721 events found
Event IDs: [  1   8  10  11  12  13  16  18  76  78 106 108]
Events detected!
number of events:  721
True
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Number of omissions and responses do not match!
Number of omissions and respons

#### Plot raw data to check the triggers

In [ ]:
import mne.viz
import matplotlib as plt
#%matplotlib inline
%matplotlib tk
#fig = mne.viz.plot_events(events=events101)

#mne.viz.plot_raw(raw=raw_appliedICA, events=events101)

In [6]:
# Concatenate epochs from all blocks:
epoch_grad = mne.concatenate_epochs(epoch_grad_all, add_offset=True)
epoch_mag = mne.concatenate_epochs(epoch_mag_all, add_offset=True)

Not setting metadata
Not setting metadata
917 matching events found
No baseline correction applied
0 bad epochs dropped
Not setting metadata
Not setting metadata
917 matching events found
No baseline correction applied
0 bad epochs dropped


In [ ]:
# Save epoched data

if tmin == -0.1:
    print('Here')
    epoch_grad.save(filep + filename[0:len(filename) - 4] + "_grad_nobase-epochs_afterICA_2.fif", overwrite=True)
    epoch_mag.save(filep + filename[0:len(filename) - 4] + "_mag_nobase-epochs_afterICA_2.fif", overwrite=True)

else:
    if tmax == 0.6:
        epoch_grad.save(filep + filename[0:len(filename) - 4] + "_grad_nobase-epochs_afterICA-elongated_2.fif", overwrite=True)
        epoch_mag.save(filep + filename[0:len(filename) - 4] + "_mag_nobase-epochs_afterICA-elongated_2.fif", overwrite=True)
    else:
        epoch_grad.save(filep + filename[0:len(filename) - 4] + "_grad_nobase-epochs_afterICA-elongated_2.fif", overwrite=True)
        epoch_mag.save(filep + filename[0:len(filename) - 4] + "_mag_nobase-epochs_afterICA-elongated_2.fif", overwrite=True)



In [ ]:
# Plot Real Events
epoch_mag['living_real_8', 'living_real_9', 'living_real_10', 'object_real_8', 'object_real_9', 'object_real_10'].crop(tmin=-0.1, tmax=0.6).average().plot(ylim=dict(mag=[-400,400]))

In [ ]:
epoch_mag

In [ ]:
# Plot Omissions
omission_events = [e for e in epoch_mag.event_id if 'real' not in e]
print(omission_events)

epoch_mag[omission_events].average().plot(ylim=dict(mag=[-400,400]))

#### Extract Confidence ratings and save them in a separate file

In [ ]:
print('Number of confidence responses: ', len(all_conf))

if s_id == '38' or s_id == '39' or s_id == '40' or s_id == '41':
    conf_filename = filep + '\\P' + s_id + '\\' + s_id + '_confValues_all'   
else:
    conf_filename = filep + '\\S' + s_id + '\\' + s_id + '_confValues_all'
np.save(conf_filename, all_conf)

In [ ]:
# Check if number of trials and number of values in confidence (real sounds (conf val = None) included)
print(len(epoch_mag))
print(len(all_conf))

In [ ]:
epoch_mag

In [ ]:
# Check data of which confidence ratings exist! There should only be OMISSIONS trials!!! 
ind = [i for i in range(len(all_conf)) if all_conf[i] != None]
epoch_mag[ind]

In [7]:
epochs_mag_old = mne.read_epochs(filep + filename[0:len(filename) - 4] + "_mag_nobase-epochs_afterICA.fif")

Reading ..\Data\MEG_Data\Data=-0.1_0.6\S33\Block_2_tsss_mc_trans_mag_nobase-epochs_afterICA.fif ...


<ipython-input-7-f6183b367fcf>:1: RuntimeWarning: This filename (..\Data\MEG_Data\Data=-0.1_0.6\S33\Block_2_tsss_mc_trans_mag_nobase-epochs_afterICA.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_mag_old = mne.read_epochs(filep + filename[0:len(filename) - 4] + "_mag_nobase-epochs_afterICA.fif")


    Found the data of interest:
        t =    -100.00 ...     600.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
917 matching events found
No baseline correction applied
0 projection items activated


In [8]:
epochs_mag_old

<EpochsFIF |  917 events (all good), -0.1 - 0.6 sec, baseline off, ~505.4 MB, data loaded,
 'living_omission_10_corr': 29
 'living_omission_10_incorr': 1
 'living_omission_8_corr': 28
 'living_omission_8_incorr': 2
 'living_omission_9_corr': 18
 'living_omission_9_incorr': 12
 'living_real_10': 120
 'living_real_8': 126
 'living_real_9': 121
 'object_omission_10_corr': 27
 'object_omission_10_incorr': 3
 'object_omission_8_corr': 23
 'object_omission_8_incorr': 7
 'object_omission_9_corr': 21
 'object_omission_9_incorr': 8
 'object_real_10': 119
 'object_real_8': 129
 'object_real_9': 123>

In [11]:
counter = 0
for i in range(epoch_mag.get_data().shape[0]):
    if epochs_mag_old.get_data()[i,:,:].all() == epoch_mag.get_data()[i,:,:].all():
        counter += 1
        
print(counter)


917


In [12]:
epoch_mag.get_data().shape

(917, 102, 701)

In [13]:
epochs_mag_old.get_data()[0,:,:]

array([[ 1.01046433e-13,  9.23534553e-14,  8.45385696e-14, ...,
         3.13215976e-14,  3.47286460e-14,  3.96705908e-14],
       [ 1.35664992e-13,  1.27176587e-13,  1.22176017e-13, ...,
        -1.40123191e-14, -1.34855176e-15,  1.17234696e-14],
       [ 1.33537035e-13,  1.08108247e-13,  8.50346504e-14, ...,
        -1.03023122e-13, -9.94698179e-14, -9.12950916e-14],
       ...,
       [-2.31818953e-14, -3.76697795e-14, -5.08296461e-14, ...,
        -7.96974508e-14, -9.03326186e-14, -1.02363233e-13],
       [-5.11236076e-14, -6.55550592e-14, -7.87036070e-14, ...,
        -5.55426890e-14, -7.03001235e-14, -8.64976576e-14],
       [-1.26623124e-13, -1.39161328e-13, -1.50282620e-13, ...,
        -4.07010225e-14, -5.43063329e-14, -7.03809240e-14]])

In [14]:
epoch_mag.get_data()[0,:,:]

array([[ 1.01046431e-13,  9.23534520e-14,  8.45385731e-14, ...,
         3.13215969e-14,  3.47286456e-14,  3.96705906e-14],
       [ 1.35664993e-13,  1.27176582e-13,  1.22176019e-13, ...,
        -1.40123186e-14, -1.34855175e-15,  1.17234701e-14],
       [ 1.33537038e-13,  1.08108247e-13,  8.50346533e-14, ...,
        -1.03023122e-13, -9.94698157e-14, -9.12950904e-14],
       ...,
       [-2.31818963e-14, -3.76697805e-14, -5.08296485e-14, ...,
        -7.96974514e-14, -9.03326190e-14, -1.02363230e-13],
       [-5.11236093e-14, -6.55550594e-14, -7.87036056e-14, ...,
        -5.55426903e-14, -7.03001239e-14, -8.64976569e-14],
       [-1.26623127e-13, -1.39161325e-13, -1.50282620e-13, ...,
        -4.07010237e-14, -5.43063347e-14, -7.03809236e-14]])